In [1]:
import duckdb
import os

# Get Wasabi credentials from environment
wasabi_endpoint = os.getenv('WASABI_ENDPOINT', 's3.us-east-2.wasabisys.com')
wasabi_access_key = os.getenv('WASABI_ACCESS_KEY')
wasabi_secret_key = os.getenv('WASABI_SECRET_KEY')
bucket_name = os.getenv('WASABI_BUCKET_NAME')

# Create DuckDB connection
con = duckdb.connect()

# Configure S3 settings
con.execute(f"""
    SET s3_endpoint='{wasabi_endpoint}';
    SET s3_access_key_id='{wasabi_access_key}';
    SET s3_secret_access_key='{wasabi_secret_key}';
    SET s3_url_style='path';
    SET preserve_insertion_order = false;
    SET enable_progress_bar = true;
""")

print("✓ DuckDB configured with S3 credentials")
print(f"Endpoint: {wasabi_endpoint}")
print(f"Access Key ID: {wasabi_access_key}")
print(f"Secret Access Key: {wasabi_secret_key}")
print('Bucket Name: ', bucket_name, '\n\n')


✓ DuckDB configured with S3 credentials
Endpoint: s3.us-east-2.wasabisys.com
Access Key ID: 60DLEZ81XUCI9ZNG8M3F
Secret Access Key: PuOySWLzPCtuts4IfXzWzdHRBefvGKgD0laHOb4B
Bucket Name:  dfscrunch-data-lake 




In [2]:
SPORT = "NFL"  # e.g., "NFL", "NBA", etc.
DATE = "2025-10-02"  # Format: YYYY-MM-DD
GAME_TYPE = "dk_single_game"  # e.g., "classic", "dk_single_game", etc.

# Maximum JSON object size (default is 16MB, we're setting to 64MB)
MAX_OBJECT_SIZE = 67108864  # 64mb in bytes

# Construct path
draft_groups_path = f"s3://{bucket_name}/staging/{SPORT}/draft_groups/{DATE}/data.json.gz"

In [40]:
df = con.execute(
    f"""
    SELECT
          "contest-sources" ->> 'id' as contest_source_id,
          "contest-sources" ->> 'source_name' as source_name,
          "contest-sources" ->> 'display_name' as display_name,
          "contest-sources" ->> 'short_name' as short_name,
          "contest-sources" ->> 'is_primary' as is_primary,
          "contest-sources" ->> 'is_active' as is_active,
          -- Draft group fields (flattened)
          dg.unnest ->> 'id' as draft_group_id,
          dg.unnest ->> 'contest_start_date',
          dg.unnest ->> 'contest_end_date',
          dg.unnest ->> 'contest_suffix',
          dg.unnest ->> 'draft_group_id' as draft_group_reference_id,
          dg.unnest ->> 'game_count',
          dg.unnest ->> 'is_simlabs',
          dg.unnest -> 'sport' ->> 'id' as sport_id,
          dg.unnest -> 'sources' ->> 'id' as source_id
      FROM read_json_auto('{draft_groups_path}', maximum_object_size={MAX_OBJECT_SIZE}),
        unnest("contest-sources".draft_groups) as dg
    """
).df()

In [41]:
df

,contest_source_id,source_name,display_name,short_name,is_primary,is_active,draft_group_id,(dg.unnest ->> 'contest_start_date'),(dg.unnest ->> 'contest_end_date'),(dg.unnest ->> 'contest_suffix'),draft_group_reference_id,(dg.unnest ->> 'game_count'),(dg.unnest ->> 'is_simlabs'),sport_id,source_id
0,4,draftkings.com,DraftKings,dk,false,true,126738,2025-10-05T13:00:00,0001-01-01T00:00:00,,134675,10,false,1,None
1,8,draftkings.com showdown,DK Showdown (CPT),,false,true,126878,2025-10-06T20:15:00,0001-01-01T00:00:00,(KC @ JAX),134887,1,false,1,None
2,8,draftkings.com showdown,DK Showdown (CPT),,false,true,126766,2025-10-02T20:15:00,0001-01-01T00:00:00,(SF @ LAR),134423,1,false,1,None
